In [1]:
import pandas as pd
from glob import glob
from experiments_to_run import *

In [2]:
paths = glob("../results/metrics*")

df = []
for path in paths:
    tmp_df = pd.read_pickle(path)
    df.append(tmp_df)
df = pd.concat(df)
df

,target,config,model,hyperparams,fold,model_obj,MSE,R2,MAPE,RMSE,NRMSE,model_name
0,CSE,Conf3,SVR,"{'gamma': 1e-05, 'C': 10}",0,"SVR(C=10, gamma=1e-05)",1.080652,0.242382,3.034680,1.039544,62.121967,SVR
1,CSE,Conf3,SVR,"{'gamma': 1e-05, 'C': 10}",1,"SVR(C=10, gamma=1e-05)",1.510847,0.110722,1.018254,1.229165,75.279267,SVR
2,CSE,Conf3,SVR,"{'gamma': 1e-05, 'C': 10}",2,"SVR(C=10, gamma=1e-05)",1.385368,0.021850,0.668871,1.177017,72.531215,SVR
3,CSE,Conf3,SVR,"{'gamma': 1e-05, 'C': 10}",3,"SVR(C=10, gamma=1e-05)",1.024837,-0.078422,2.294448,1.012342,55.846892,SVR
4,CSE,Conf3,SVR,"{'gamma': 1e-05, 'C': 10}",4,"SVR(C=10, gamma=1e-05)",1.504243,-0.013059,1.363649,1.226476,71.465827,SVR
...,...,...,...,...,...,...,...,...,...,...,...,...
5,CS,Conf1,MLP,"{'max_iter': 50, 'learning_rate_init': 0.01, '...",5,"MLPRegressor(batch_size=10, hidden_layer_sizes...",3999.314906,-0.008811,4.985367,63.240137,82.953230,MLPRegressor
6,CS,Conf1,MLP,"{'max_iter': 50, 'learning_rate_init': 0.01, '...",6,"MLPRegressor(batch_size=10, hidden_layer_sizes...",3266.357367,0.047601,1.248889,57.152055,72.238464,MLPRegressor
7,CS,Conf1,MLP,"{'max_iter': 50, 'learning_rate_init': 0.01, '...",7,"MLPRegressor(batch_size=10, hidden_layer_sizes...",3770.076482,0.161533,7.455843,61.400949,83.170102,MLPRegressor
8,CS,Conf1,MLP,"{'max_iter': 50, 'learning_rate_init': 0.01, '...",8,"MLPRegressor(batch_size=10, hidden_layer_sizes...",2919.004235,0.152228,6.145434,54.027810,67.432255,MLPRegressor


In [3]:
metrics = ["R2", "MAPE", "RMSE", "NRMSE"]
mean_df = df.groupby(["target", "config", "model"]).agg("mean")[metrics]
std_df = df.groupby(["target", "config", "model"]).agg("std")[metrics]

display(mean_df)
display(std_df)

R2      MAPE       RMSE      NRMSE
target config model                                             
CS     Conf1  CatBoost  0.283532  2.888960  51.153450  66.131979
              GBDT      0.247073  3.398494  52.503845  67.898500
              KNN       0.231872  4.123735  52.958285  68.449389
              MLP       0.117044  3.792172  56.855548  73.523343
              RF        0.300984  2.915477  50.525215  65.324238
              SVR       0.196813  3.493187  54.278858  70.234684
              XGBoost   0.210190  2.308685  53.485490  69.091973
       Conf2  CatBoost  0.294936  3.015568  50.709448  65.549055
              GBDT      0.253469  3.410568  52.272047  67.595868
              KNN       0.258284  4.007496  52.055210  67.298933
              MLP       0.180579  3.771066  54.771509  70.857588
              RF        0.302350  3.033192  50.479633  65.264142
              SVR       0.212509  3.280024  53.751613  69.560078
              XGBoost   0.230120  2.358447  52.803490  68.187315
       Conf3  CatBoost  0.695410  1.456096  33.398132  43.219761
              GBDT      0.667843  0.692878  34.836990  45.049687
              KNN       0.233576  4.094910  52.899754  68.373555
              MLP       0.605455  1.018895  37.938869  49.060462
              RF        0.692314  1.441102  33.559760  43.428975
              SVR       0.270398  1.905427  51.536411  66.629524
              XGBoost   0.648970  0.707795  35.741998  46.201048
       Conf4  CatBoost  0.695432  1.518070  33.400298  43.221228
              GBDT      0.646993  0.692483  35.950195  46.506550
              KNN       0.259664  4.005691  52.008826  67.239550
              MLP       0.569950  0.626697  39.407326  51.015711
              RF        0.668085  1.724044  34.861444  45.110180
              SVR       0.216963  3.249409  53.600234  69.364593
              XGBoost   0.664891  0.710421  35.021918  45.297539
CSE    Conf1  CatBoost  0.271997  1.904110   1.005249  59.257498
              GBDT      0.136450  1.848094   1.090668  64.218069
              KNN      -0.027481  2.618994   1.191079  70.133170
              MLP      -0.078865  2.554188   1.224648  72.181197
              RF        0.286125  1.781647   0.995246  58.673188
              SVR      -0.051295  2.265032   1.207862  71.171426
              XGBoost   0.183284  1.726899   1.060524  62.459239
       Conf2  CatBoost  0.234702  1.949768   1.029314  60.662784
              GBDT      0.220151  2.108656   1.041079  61.368465
              KNN       0.056047  2.421659   1.144183  67.412214
              MLP      -0.034228  2.702805   1.198987  70.666972
              RF        0.278207  1.837294   1.001704  59.065587
              SVR      -0.019651  2.319813   1.186498  69.866031
              XGBoost   0.178408  1.785789   1.063945  62.667573
       Conf3  CatBoost  0.422339  1.445491   0.896515  52.884083
              GBDT      0.314708  1.271793   0.973174  57.331939
              KNN      -0.021976  2.610048   1.188103  69.961224
              MLP      -0.035033  2.704918   1.199532  70.700663
              RF        0.421078  1.340473   0.897487  52.929781
              SVR       0.017032  1.960457   1.164953  68.590066
              XGBoost   0.322897  1.263129   0.966912  56.976030
       Conf4  CatBoost  0.403671  1.427673   0.909392  53.596797
              GBDT      0.308743  1.337390   0.975764  57.476666
              KNN       0.058160  2.419708   1.142982  67.341915
              MLP      -0.030763  2.716778   1.197637  70.593658
              RF        0.412746  1.413925   0.904265  53.332962
              SVR       0.056330  2.022878   1.144673  67.456680
              XGBoost   0.310091  1.302538   0.977084  57.563014

R2      MAPE      RMSE     NRMSE
target config model                                           
CS     Conf1  CatBoost  0.061787  1.486986  3.640261  6.335449
              GBDT      0.044895  1.780466  4.039115  7.030953
              KNN       0.054102  2.655204  3.133358  5.743371
              MLP       0.097635  2.644462  5.584500  8.780210
              RF        0.065886  1.563907  3.826947  6.541386
              SVR       0.046021  2.191900  4.817634  8.281179
              XGBoost   0.124677  1.026404  3.432378  5.548027
       Conf2  CatBoost  0.065710  1.659952  3.294003  5.874297
              GBDT      0.048063  1.788380  4.001461  6.951798
              KNN       0.054286  2.580175  3.461067  6.214181
              MLP       0.089738  2.403819  5.422021  8.700174
              RF        0.055827  1.659695  3.511197  6.218238
              SVR       0.056129  2.056653  5.112650  8.618153
              XGBoost   0.110633  1.053525  2.168212  3.796807
       Conf3  CatBoost  0.047933  0.753634  4.269045  6.524646
              GBDT      0.047532  0.222152  3.815700  5.832355
              KNN       0.054013  2.625089  3.135989  5.739962
              MLP       0.061088  0.326209  4.080238  6.260941
              RF        0.046282  0.731537  4.115313  6.377306
              SVR       0.102998  1.316621  4.598656  7.411490
              XGBoost   0.041715  0.263010  2.334271  4.132633
       Conf4  CatBoost  0.045239  0.807254  4.128440  6.353724
              GBDT      0.052870  0.212123  4.374112  6.648756
              KNN       0.053989  2.581680  3.489057  6.246182
              MLP       0.120402  0.242617  6.142349  9.130475
              RF        0.047305  0.917558  4.177218  6.497283
              SVR       0.055783  2.031584  5.107500  8.607182
              XGBoost   0.032280  0.227858  3.257707  5.377477
CSE    Conf1  CatBoost  0.092929  0.603685  0.099987  7.407509
              GBDT      0.130422  0.557704  0.076898  5.947610
              KNN       0.120143  1.124444  0.064793  5.705512
              MLP       0.146923  0.881335  0.133772  9.454123
              RF        0.091406  0.503535  0.094717  7.185745
              SVR       0.092027  1.016221  0.091382  7.368613
              XGBoost   0.132512  0.529924  0.087662  6.551630
       Conf2  CatBoost  0.109222  0.647717  0.095864  7.192749
              GBDT      0.064301  0.679648  0.085257  6.882605
              KNN       0.079826  1.055380  0.079799  6.677735
              MLP       0.054721  0.935305  0.085521  7.321282
              RF        0.078191  0.553155  0.095985  7.367692
              SVR       0.133998  0.823053  0.081816  6.479546
              XGBoost   0.122492  0.582654  0.070297  5.870035
       Conf3  CatBoost  0.089195  0.420374  0.114414  8.066892
              GBDT      0.104302  0.311975  0.096521  6.898222
              KNN       0.116142  1.125752  0.065293  5.759647
              MLP       0.052878  0.933909  0.085940  7.351868
              RF        0.076560  0.321777  0.103542  7.472240
              SVR       0.128650  0.860624  0.083086  6.297357
              XGBoost   0.112740  0.305146  0.099411  7.162737
       Conf4  CatBoost  0.079201  0.408901  0.093146  6.750793
              GBDT      0.130842  0.355198  0.108848  7.449251
              KNN       0.077919  1.055598  0.079628  6.666581
              MLP       0.042326  0.941857  0.090643  7.567174
              RF        0.071010  0.366441  0.101325  7.406025
              SVR       0.071988  0.802458  0.083289  6.964362
              XGBoost   0.096601  0.329503  0.092619  6.723627

In [4]:
for target_var in TARGETS:
    
    tmp = df[(df.target == target_var)]\
      [["target", "config", "model", "hyperparams"]]\
      .groupby(["config", "model"]).agg({
        "hyperparams":"first",
      }).reset_index()

    tmp.hyperparams = tmp.hyperparams.apply(
        lambda x: "\n".join([f"{k}: {v}" for k,v in x.items()]) if x is not None else ""
    )
    
    gb = tmp.groupby("model")
    new_tmp = []
    for m,g in gb:
        new_tmp.append(g.drop(columns=["model"]).rename(columns={"hyperparams":m}).set_index("config").T)
    
    tmp = pd.concat(new_tmp).reset_index().rename(columns={"index":"Model"})
    
    print("     "+target_var)
    print("results exported to", f"../results/hyperparams_{target_var}.[txt/xlsx]")
    tmp.to_markdown(f"../results/hyperparams_{target_var}.txt", index=False, tablefmt="fancy_grid")
    tmp.to_excel(f"../results/hyperparams_{target_var}.xlsx", index=False)
    
    print("results exported to", f"../figures_and_tables/table_appendix_hyperparameters_{target_var}.[txt/xlsx/csv]")
    tmp.to_markdown(f"../figures_and_tables/table_appendix_hyperparameters_{target_var}.txt", index=False, tablefmt="fancy_grid")
    tmp.to_excel(f"../figures_and_tables/table_appendix_hyperparameters_{target_var}.xlsx", index=False)
    tmp.to_csv(f"../figures_and_tables/table_appendix_hyperparameters_{target_var}.csv", index=False)


     CS
results exported to ../results/hyperparams_CS.[txt/xlsx]
results exported to ../figures_and_tables/table_appendix_hyperparameters_CS.[txt/xlsx/csv]
     CSE
results exported to ../results/hyperparams_CSE.[txt/xlsx]
results exported to ../figures_and_tables/table_appendix_hyperparameters_CSE.[txt/xlsx/csv]
